In [13]:
from lib.network import PoseNetRGBOnly, PoseNet
from lib.loss import Loss
from lib.utils import setup_logger
from datasets.linemod.dataset import PoseDataset as PoseDataset_linemod
import torch
import os
from torch import optim
import numpy as np
import time
from torch.autograd import Variable
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import warnings
warnings.filterwarnings("ignore")

# Load data

In [3]:
num_objects = 5
num_points = 500
outf = 'trained_models/linemod'
log_dir = 'experiments/logs/linemod'
repeat_epoch = 20
decay_margin = 0.016
decay_start = False
lr_rate = 0.3
w_rate = 0.3

is_RGBD = True
input_depth=True
add_depth_to_output=True
true_depth=False

In [4]:
dataset_root = "./datasets/linemod/Linemod_preprocessed"
noise_trans = 0.03
refine_start = False
decay_start = False


dataset = PoseDataset_linemod('train', 
                              num_points, 
                              True, 
                              dataset_root, 
                              noise_trans, 
                              refine_start, 
                              use_true_depth=False)

dataloader = torch.utils.data.DataLoader(dataset, 
                                         batch_size=1, 
                                         shuffle=True, 
                                         num_workers=4)
test_dataset = PoseDataset_linemod('test', 
                                   num_points, 
                                   False, 
                                   dataset_root, 
                                   0.0, 
                                   refine_start)
testdataloader = torch.utils.data.DataLoader(test_dataset, 
                                             batch_size=1, 
                                             shuffle=False, 
                                             num_workers=4)

Object 2 buffer loaded
Object 4 buffer loaded
Object 5 buffer loaded
Object 10 buffer loaded
Object 11 buffer loaded
Object 2 buffer loaded
Object 4 buffer loaded
Object 5 buffer loaded
Object 10 buffer loaded
Object 11 buffer loaded


In [5]:
sym_list = dataset.get_sym_list()
num_points_mesh = dataset.get_num_points_mesh()

# Models

In [14]:
if is_RGBD:
    # RGBD
    estimator = PoseNet(num_points = num_points, num_obj = num_objects)
#     model = 'trained_models/linemod/pose_model_12_0.09392938120923575.pth'
else:
    estimator = PoseNetRGBOnly(num_points=num_points, num_obj = num_objects)
#     model = 'trained_models/linemod/pose_model_12_0.09392938120923575.pth'
# estimator.load_state_dict(torch.load(model))
estimator.cuda();

lr = 0.0001
optimizer = optim.Adam(estimator.parameters(), lr=lr)

# Cases
## True depth, predict depth
1. input + depth, output + depth
2. input, output + depth
3. input, output

In [15]:
criterion = Loss(num_points_mesh, sym_list)

In [16]:
start_epoch = 1
nepoch = 16
w = 0.015
batch_size = 8

In [17]:
best_test = np.Inf
print_every = 50

if start_epoch == 1:
    for log in os.listdir(log_dir):
        if '.ipyn' not in log:
            os.remove(os.path.join(log_dir, log))
st_time = time.time()

for epoch in range(start_epoch, nepoch):
    logger = setup_logger('epoch%d' % epoch, os.path.join(log_dir, 'epoch_%d_log.txt' % epoch))
    mess = 'Train time {0}'.format(time.strftime("%Hh %Mm %Ss", time.gmtime(time.time() - st_time)) + ', ' + 'Training started')
    print(mess)
    logger.info(mess)
    
    train_count = 0
    train_dis_avg = 0.0
    
    estimator.train()
    optimizer.zero_grad()

    for rep in range(repeat_epoch):
        for i, data in enumerate(dataloader, 0):
            points, choose, img, target, model_points, idx = data  

            points, choose, img, target, model_points, idx = Variable(points).cuda(), \
                                                             Variable(choose).cuda(), \
                                                             Variable(img).cuda(), \
                                                             Variable(target).cuda(), \
                                                             Variable(model_points).cuda(), \
                                                             Variable(idx).cuda()
            if is_RGBD:
                pred_r, pred_t, pred_c, emb = estimator(img, points, choose, idx)
            else:
                if not add_depth_to_output:
                    points[0, :, 2] = 0 
                pred_r, pred_t, pred_c, emb = estimator(img, choose, idx)
                
            loss, dis, new_points, new_target = criterion(pred_r, 
                                                          pred_t, 
                                                          pred_c, 
                                                          target, 
                                                          model_points, 
                                                          idx, 
                                                          points, 
                                                          w, refine_start)
            loss.backward()
            
            train_dis_avg += dis.item()
            train_count += 1
            if train_count % batch_size == 0:
                
                mess = 'Train time {0} Epoch {1} Batch {2} Frame {3} Avg_dis:{4}'.format(
                        time.strftime("%Hh %Mm %Ss", time.gmtime(time.time() - st_time)), 
                        epoch, int(train_count / batch_size), 
                        train_count, 
                        train_dis_avg / batch_size)
                logger.info(mess)
                
                optimizer.step()
                optimizer.zero_grad()
                train_dis_avg = 0
                
                
                if train_count % print_every == 0:
                    print(mess)
            
            if train_count != 0 and train_count % 500 == 0:                    
                torch.save(estimator.state_dict(), '{0}/pose_model_current.pth'.format(outf))
            
    print('>>>>>>>>----------epoch {0} train finish---------<<<<<<<<'.format(epoch))
    
    test_dis = 0.0
    test_count = 0
    estimator.eval()
    
    for j, data in enumerate(testdataloader, 0):
        points, choose, img, target, model_points, idx = data
        points, choose, img, target, model_points, idx = Variable(points).cuda(), \
                                                         Variable(choose).cuda(), \
                                                         Variable(img).cuda(), \
                                                         Variable(target).cuda(), \
                                                         Variable(model_points).cuda(), \
                                                         Variable(idx).cuda()
        
        if is_RGBD:
            pred_r, pred_t, pred_c, emb = estimator(img, points, choose, idx)
        else:
            if not add_depth_to_output:
                points[0, :, 2] = 0 
            pred_r, pred_t, pred_c, emb = estimator(img, choose, idx)
            
        _, dis, new_points, new_target = criterion(pred_r, pred_t, pred_c, 
                                                   target, 
                                                   model_points, idx, points, 
                                                   w, refine_start)
        test_dis += dis.item()
        mess = 'Test time {0} Test Frame No.{1} dis:{2}'.format(time.strftime("%Hh %Mm %Ss", time.gmtime(time.time() - st_time)), test_count, dis)
        logger.info(mess)
        
        test_count += 1
    test_dis = test_dis / test_count
    mess = 'Test time {0} Epoch {1} TEST FINISH Avg dis: {2}'.format(time.strftime("%Hh %Mm %Ss", time.gmtime(time.time() - st_time)), epoch, test_dis)
    print(mess)
    logger.info(mess)
    
    if test_dis <= best_test:
        best_test = test_dis
        torch.save(estimator.state_dict(), '{0}/pose_model_{1}_{2}.pth'.format(outf, epoch, test_dis))
        print(epoch, '>>>>>>>>----------BEST TEST MODEL SAVED---------<<<<<<<<')
        
    if best_test < decay_margin and not decay_start:
        decay_start = True
        lr *= lr_rate
        w *= w_rate
        optimizer = optim.Adam(estimator.parameters(), lr=lr)

Train time 00h 52m 41s Epoch 4 Batch 75 Frame 600 Avg_dis:0.01997908018529415
Train time 00h 52m 52s Epoch 4 Batch 100 Frame 800 Avg_dis:0.022109096287749708
Train time 00h 53m 03s Epoch 4 Batch 125 Frame 1000 Avg_dis:0.019802452879957855
Train time 00h 53m 14s Epoch 4 Batch 150 Frame 1200 Avg_dis:0.01827411272097379
Train time 00h 53m 25s Epoch 4 Batch 175 Frame 1400 Avg_dis:0.0292574146296829
Train time 00h 53m 36s Epoch 4 Batch 200 Frame 1600 Avg_dis:0.018033112690318376
Train time 00h 53m 47s Epoch 4 Batch 225 Frame 1800 Avg_dis:0.020528267021290958
Train time 00h 53m 58s Epoch 4 Batch 250 Frame 2000 Avg_dis:0.029990330571308732
Train time 00h 54m 10s Epoch 4 Batch 275 Frame 2200 Avg_dis:0.01749460364226252
Train time 00h 54m 21s Epoch 4 Batch 300 Frame 2400 Avg_dis:0.023816030472517014
Train time 00h 54m 32s Epoch 4 Batch 325 Frame 2600 Avg_dis:0.027413675794377923
Train time 00h 54m 43s Epoch 4 Batch 350 Frame 2800 Avg_dis:0.018227845313958824
Train time 00h 54m 54s Epoch 4 Batch

KeyboardInterrupt: 

In [ ]:
dataset_root = "./datasets/linemod/Linemod_preprocessed"
noise_trans = 0.03
refine_start = False
decay_start = False


dataset = PoseDataset_linemod('train', 
                              num_points, 
                              True, 
                              dataset_root, 
                              noise_trans, 
                              refine_start)

workers = 4
dataloader = torch.utils.data.DataLoader(dataset, 
                                         batch_size=1, 
                                         shuffle=True, 
                                         num_workers=workers)

In [ ]:
test_dataset = PoseDataset_linemod('test', 
                                   num_points, 
                                   False, 
                                   dataset_root, 
                                   0.0, 
                                   refine_start)
testdataloader = torch.utils.data.DataLoader(test_dataset, 
                                             batch_size=1, 
                                             shuffle=False, 
                                             num_workers=workers)

In [ ]:
sym_list = dataset.get_sym_list()
num_points_mesh = dataset.get_num_points_mesh()

In [ ]:
print('>>>>>>>>----------Dataset loaded!---------<<<<<<<<\nlength of the training set: {0}\nlength of the testing set: {1}\nnumber of sample points on mesh: {2}\nsymmetry object list: {3}'
      .format(len(dataset), 
              len(test_dataset), 
              num_points_mesh, 
              sym_list))

In [ ]:
criterion = Loss(num_points_mesh, sym_list)

In [ ]:
start_epoch = 9
nepoch = 16
w = 0.015
batch_size = 8

In [ ]:
next(iter(dataloader))[1].shape

In [ ]:
lr *= lr_rate
w *= w_rate
optimizer = optim.Adam(estimator.parameters(), lr=lr)

In [47]:
best_test = np.Inf
print_every = 50

if start_epoch == 1:
    for log in os.listdir(log_dir):
        if '.ipyn' not in log:
            os.remove(os.path.join(log_dir, log))
st_time = time.time()

for epoch in range(start_epoch, nepoch):
    logger = setup_logger('epoch%d' % epoch, os.path.join(log_dir, 'epoch_%d_log.txt' % epoch))
    mess = 'Train time {0}'.format(time.strftime("%Hh %Mm %Ss", time.gmtime(time.time() - st_time)) + ', ' + 'Training started')
    print(mess)
    logger.info(mess)
    
    train_count = 0
    train_dis_avg = 0.0
    
    estimator.train()
    optimizer.zero_grad()

    for rep in range(repeat_epoch):
        for i, data in enumerate(dataloader, 0):
            points, choose, img, target, model_points, idx = data 
            points[0, :, 2] = 0 # we don't know the z
            points, choose, img, target, model_points, idx = Variable(points).cuda(), \
                                                             Variable(choose).cuda(), \
                                                             Variable(img).cuda(), \
                                                             Variable(target).cuda(), \
                                                             Variable(model_points).cuda(), \
                                                             Variable(idx).cuda()
            
            pred_r, pred_t, pred_c, emb = estimator(img, choose, idx)
            loss, dis, _, _ = criterion(pred_r, pred_t, pred_c, target, 
                                  model_points, idx,
                                  points, w, refine_start)
            loss.backward()
            
            train_dis_avg += dis.item()
            train_count += 1
            if train_count % batch_size == 0:
                
                mess = 'Train time {0} Epoch {1} Batch {2} Frame {3} Avg_dis:{4}'.format(
                        time.strftime("%Hh %Mm %Ss", time.gmtime(time.time() - st_time)), 
                        epoch, int(train_count / batch_size), 
                        train_count, 
                        train_dis_avg / batch_size)
                logger.info(mess)
                
                optimizer.step()
                optimizer.zero_grad()
                train_dis_avg = 0
                
                
                if train_count % print_every == 0:
                    print(mess)
            
            if train_count != 0 and train_count % 500 == 0:                    
                torch.save(estimator.state_dict(), '{0}/pose_model_current.pth'.format(outf))
            
    print('>>>>>>>>----------epoch {0} train finish---------<<<<<<<<'.format(epoch))
    
    test_dis = 0.0
    test_count = 0
    estimator.eval()
    
    for j, data in enumerate(testdataloader, 0):
        points, choose, img, target, model_points, idx = data
        points[0, :, 2] = 0 # we don't know the z
        points, choose, img, target, model_points, idx = Variable(points).cuda(), \
                                                         Variable(choose).cuda(), \
                                                         Variable(img).cuda(), \
                                                         Variable(target).cuda(), \
                                                         Variable(model_points).cuda(), \
                                                         Variable(idx).cuda()
       
        pred_r, pred_t, pred_c, emb = estimator(img, choose, idx)
        _, dis, _, _ = criterion(pred_r, pred_t, pred_c, target,
                           model_points, idx, 
                           points, w, refine_start)
        test_dis += dis.item()
        mess = 'Test time {0} Test Frame No.{1} dis:{2}'.format(time.strftime("%Hh %Mm %Ss", time.gmtime(time.time() - st_time)), test_count, dis)
        logger.info(mess)
        
        test_count += 1
    test_dis = test_dis / test_count
    mess = 'Test time {0} Epoch {1} TEST FINISH Avg dis: {2}'.format(time.strftime("%Hh %Mm %Ss", time.gmtime(time.time() - st_time)), epoch, test_dis)
    print(mess)
    logger.info(mess)
    
    if test_dis <= best_test:
        best_test = test_dis
        torch.save(estimator.state_dict(), '{0}/pose_model_{1}_{2}.pth'.format(outf, epoch, test_dis))
        print(epoch, '>>>>>>>>----------BEST TEST MODEL SAVED---------<<<<<<<<')
        
    if best_test < decay_margin and not decay_start:
        decay_start = True
        lr *= lr_rate
        w *= w_rate
        optimizer = optim.Adam(estimator.parameters(), lr=lr)

Train time 01h 43m 31s Epoch 15 Batch 275 Frame 2200 Avg_dis:0.0177555350237526
Train time 01h 43m 42s Epoch 15 Batch 300 Frame 2400 Avg_dis:0.017905657296068966
Train time 01h 43m 54s Epoch 15 Batch 325 Frame 2600 Avg_dis:0.021497412235476077
Train time 01h 44m 06s Epoch 15 Batch 350 Frame 2800 Avg_dis:0.024940106726717204
Train time 01h 44m 18s Epoch 15 Batch 375 Frame 3000 Avg_dis:0.015354499570094049
Train time 01h 44m 29s Epoch 15 Batch 400 Frame 3200 Avg_dis:0.021991423796862364
Train time 01h 44m 40s Epoch 15 Batch 425 Frame 3400 Avg_dis:0.023702603532001376
Train time 01h 44m 53s Epoch 15 Batch 450 Frame 3600 Avg_dis:0.02132924017496407
Train time 01h 45m 07s Epoch 15 Batch 475 Frame 3800 Avg_dis:0.019769376260228455
Train time 01h 45m 20s Epoch 15 Batch 500 Frame 4000 Avg_dis:0.024137724423781037
Train time 01h 45m 33s Epoch 15 Batch 525 Frame 4200 Avg_dis:0.021446194732561707
Train time 01h 45m 46s Epoch 15 Batch 550 Frame 4400 Avg_dis:0.020798002369701862
Train time 01h 45m 